In [1]:
# # 01 — Load and Inspect

# This notebook loads the CSV file using the project preprocessing module.
# It inspects:
# • metadata
# • frame indices
# • timestamps
# • position arrays
# quaternion arrays

In [2]:
import os
import numpy as np
import pandas as pd

# Make sure src/ is on the Python path
import sys
# Check if we're in notebooks/ dir or project root
if os.path.basename(os.getcwd()) == 'notebooks':
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
else:
    PROJECT_ROOT = os.path.abspath(os.getcwd())
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

from pipeline_config import CONFIG
from preprocessing import parse_optitrack_csv

print("Paths configured.")

Paths configured.


In [3]:
# Path loaded from config.yaml
DATA_DIR = os.path.join(PROJECT_ROOT, CONFIG['data_dir'])
CSV_PATH = os.path.join(DATA_DIR, CONFIG['current_csv'])

print("CSV path:", CSV_PATH)

CSV path: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\data\734/T1/734_T1_P1_R2_Take 2025-12-01 02.32.02 PM.csv


In [4]:
import json

SCHEMA_PATH = os.path.join(PROJECT_ROOT, "config", "skeleton_schema.json")

if os.path.exists(SCHEMA_PATH):
    with open(SCHEMA_PATH, "r") as f:
        schema = json.load(f)
    print("Schema loaded:", SCHEMA_PATH)
else:
    schema = None
    print("Schema file not found — you must create or provide one.")

Schema loaded: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\config\skeleton_schema.json


In [5]:
frame_idx, time_s, pos_mm, q_global, loader_report = parse_optitrack_csv(
    CSV_PATH,
    schema
)

print("CSV parsed successfully.")

CSV parsed successfully.


In [6]:
# # Run this in a new cell to inspect the raw file structure
# with open(CSV_PATH, 'r', encoding='utf-8', errors='ignore') as f:
#     print("--- FIRST 10 LINES OF CSV ---")
#     for i in range(10):
#         print(repr(f.readline())) # repr() shows us hidden characters like \t or \r

In [7]:
import pprint
pprint.pprint(loader_report)

{'calibration': {'export_date': None,
                 'optitrack_version': 'unknown',
                 'pointer_tip_rms_error_mm': None,
                 'wand_error_mm': None},
 'data_quality': {'nan_position_percent': '0.00%',
                  'nan_rotation_percent': '0.00%'},
 'duration_sec': 254.275,
 'file_path': 'c:\\Users\\drorh\\OneDrive - '
              'Mobileye\\Desktop\\gaga\\data\\734/T1/734_T1_P1_R2_Take '
              '2025-12-01 02.32.02 PM.csv',
 'fps_estimated': 120.00480019200238,
 'segments_expected': 51,
 'segments_found_count': 51,
 'segments_found_list': ['Hips',
                         'Spine',
                         'Spine1',
                         'Neck',
                         'Head',
                         'LeftShoulder',
                         'LeftArm',
                         'LeftForeArm',
                         'LeftHand',
                         'LeftHandThumb1',
                         'LeftHandThumb2',
                         'Le

In [8]:
print("Frame count:", len(frame_idx))
print("Time array shape:", time_s.shape)
print("Positions array shape:", pos_mm.shape)   # (T, J, 3)
print("Quaternions array shape:", q_global.shape)  # (T, J, 4)

Frame count: 30514
Time array shape: (30514,)
Positions array shape: (30514, 51, 3)
Quaternions array shape: (30514, 51, 4)


In [9]:
print("First 10 timestamps:", time_s[:10])
print("Last timestamp:", time_s[-1])
print("Effective FPS estimate:", 1.0 / np.median(np.diff(time_s)))

First 10 timestamps: [0.       0.008333 0.016667 0.025    0.033333 0.041667 0.05     0.058333
 0.066667 0.075   ]
Last timestamp: 254.275
Effective FPS estimate: 120.00480019200238


In [10]:
pos_missing = np.isnan(pos_mm).sum()
quat_missing = np.isnan(q_global).sum()

print("Missing position values:", pos_missing)
print("Missing quaternion values:", quat_missing)

Missing position values: 0
Missing quaternion values: 0


In [11]:
print("First frame positions (mm):")
print(pos_mm[0])

print("\nFirst frame quaternions (xyzw):")
print(q_global[0])

First frame positions (mm):
[[ 188.376511  878.021973  299.115051]
 [ 188.059982  959.145996  298.36615 ]
 [ 187.579788 1203.069336  302.008606]
 [ 187.706284 1459.543945  312.035919]
 [ 182.816772 1601.108643  343.392273]
 [ 225.303207 1398.218506  306.661591]
 [ 378.694244 1423.983398  279.54538 ]
 [ 697.662415 1360.676392  273.566101]
 [ 884.768188 1324.007202  266.995605]
 [ 920.512085 1301.900391  301.286133]
 [ 938.717407 1320.657837  315.551056]
 [ 961.633484 1331.7323    315.681702]
 [ 980.288513 1356.8302    306.408081]
 [1023.111816 1349.16272   295.670319]
 [1035.358765 1332.255737  287.53714 ]
 [ 978.776062 1360.892456  280.495728]
 [1025.897217 1352.682617  268.588501]
 [1041.163574 1332.286865  259.997406]
 [ 972.095459 1362.758789  254.768463]
 [1015.230286 1354.501953  245.87233 ]
 [1028.534058 1337.009644  241.51207 ]
 [ 965.402039 1364.65979   228.819733]
 [ 999.851624 1357.927612  221.541214]
 [1010.754578 1343.964966  218.813492]
 [ 149.935776 1398.161865  309.05960

In [12]:
from skeleton_defs import SKELETON_HIERARCHY

def validate_hierarchy_and_build_map(found_joints_list, strict_mode=True):
    """
    Quality Gate:
    1. Checks if ALL joints defined in SKELETON_HIERARCHY exist in the data.
    2. Builds the final kinematics mapping dictionary.
    3. Throws a CRITICAL ERROR if strict_mode is True and any joint is missing.
    """
    
    print(f"\n{'='*20} HIERARCHY VALIDATION REPORT {'='*20}")
    print(f"Strict Mode: {strict_mode}\n")
    
    kinematics_map = {}
    missing_joints = []
    
    
    found_set = set(found_joints_list)
    
 
    for segment, info in SKELETON_HIERARCHY.items():
        parent = info['parent']
        angle_name = info['angle_name']
        
       
        if segment not in found_set:
            print(f"[FAIL] ❌ Segment Missing: {segment}")
            missing_joints.append(segment)
            continue
            
       
        if parent is not None and parent not in found_set:
            print(f"[FAIL] ❌ Parent Missing for {segment}: {parent}")
            missing_joints.append(f"{segment}_parent_{parent}")
            continue

        # 3. בניית המיפוי להמשך
        status = "Global" if parent is None else f"Rel: {parent}"
        print(f"[PASS] ✅ {segment:<15} -> {status:<15} => Output: '{angle_name}'")
        
        kinematics_map[segment] = {
            "parent": parent,
            "angle_name": angle_name,
            "is_global": (parent is None)
        }

    print(f"\n{'='*60}")
    
 
    if len(missing_joints) > 0:
        error_msg = f"CRITICAL VALIDATION ERROR: Missing {len(missing_joints)} required joints: {missing_joints}"
        if strict_mode:
            raise ValueError(error_msg)
        else:
            print(f"WARNING: {error_msg}")
    else:
        print("SUCCESS: Full skeleton hierarchy validated successfully.")
        
    return kinematics_map


try:
    kinematics_targets = validate_hierarchy_and_build_map(
        loader_report['segments_found_list'], 
        strict_mode=True 
    )
    print("\nKinematics Map Created Ready for Processing.")
except ValueError as e:
    print(f"\n⛔ STOPPING EXECUTION: {e}")


==================== HIERARCHY VALIDATION REPORT ====================
Strict Mode: True

[PASS] ✅ Hips            -> Global          => Output: 'Pelvis_Global_Orientation (World Space)'
[PASS] ✅ Spine           -> Rel: Hips       => Output: 'Lumbar_Angle'
[PASS] ✅ Spine1          -> Rel: Spine      => Output: 'Thoracic_Angle'
[PASS] ✅ Neck            -> Rel: Spine1     => Output: 'Neck_Base_Angle'
[PASS] ✅ Head            -> Rel: Neck       => Output: 'Head_Angle'
[PASS] ✅ LeftUpLeg       -> Rel: Hips       => Output: 'LeftHip_Angle'
[PASS] ✅ LeftLeg         -> Rel: LeftUpLeg  => Output: 'LeftKnee_Angle'
[PASS] ✅ LeftFoot        -> Rel: LeftLeg    => Output: 'LeftAnkle_Angle'
[PASS] ✅ LeftToeBase     -> Rel: LeftFoot   => Output: 'LeftToe_Angle'
[PASS] ✅ RightUpLeg      -> Rel: Hips       => Output: 'RightHip_Angle'
[PASS] ✅ RightLeg        -> Rel: RightUpLeg => Output: 'RightKnee_Angle'
[PASS] ✅ RightFoot       -> Rel: RightLeg   => Output: 'RightAnkle_Angle'
[PASS] ✅ RightToeBase   

In [13]:
# --- Save parsed data for Step 02 ---
import os

# 1. Create Output Directory
DERIV_01 = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_01_parse")
os.makedirs(DERIV_01, exist_ok=True)

# 2. Convert raw arrays to DataFrame for saving
# We need to flatten the 3D arrays (Frames, Joints, 3/4) into 2D columns
flattened_data = {
    "frame_idx": frame_idx,
    "time_s": time_s
}

joint_names = schema['joint_names']

for i, name in enumerate(joint_names):
    # Save Position (X, Y, Z)
    flattened_data[f"{name}__px"] = pos_mm[:, i, 0]
    flattened_data[f"{name}__py"] = pos_mm[:, i, 1]
    flattened_data[f"{name}__pz"] = pos_mm[:, i, 2]
    
    # Save Rotation (X, Y, Z, W)
    flattened_data[f"{name}__qx"] = q_global[:, i, 0]
    flattened_data[f"{name}__qy"] = q_global[:, i, 1]
    flattened_data[f"{name}__qz"] = q_global[:, i, 2]
    flattened_data[f"{name}__qw"] = q_global[:, i, 3]

df_export = pd.DataFrame(flattened_data)

# 3. Save to Parquet
run_id = os.path.splitext(os.path.basename(CSV_PATH))[0]
out_path = os.path.join(DERIV_01, f"{run_id}__parsed_run.parquet")

df_export.to_parquet(out_path, index=False)

print(f"✅ Data saved successfully to:\n{out_path}")

✅ Data saved successfully to:
c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_01_parse\734_T1_P1_R2_Take 2025-12-01 02.32.02 PM__parsed_run.parquet


In [14]:
import pandas as pd
import json
import os
import numpy as np

# --- 1. Build DataFrame ---
# שימוש ב-joints_found מתוך ה-loader_report
joints_found = loader_report['segments_found_list']

flattened_data = {
    'frame_idx': frame_idx,
    'time_s': time_s
}

for j_idx, joint_name in enumerate(joints_found):
    # Position (mm) - Mapping to __px, __py, __pz
    flattened_data[f'{joint_name}__px'] = pos_mm[:, j_idx, 0]
    flattened_data[f'{joint_name}__py'] = pos_mm[:, j_idx, 1]
    flattened_data[f'{joint_name}__pz'] = pos_mm[:, j_idx, 2]
    # Rotation (Quaternion) - Mapping to __qx, __qy, __qz, __qw
    flattened_data[f'{joint_name}__qx'] = q_global[:, j_idx, 0]
    flattened_data[f'{joint_name}__qy'] = q_global[:, j_idx, 1]
    flattened_data[f'{joint_name}__qz'] = q_global[:, j_idx, 2]
    flattened_data[f'{joint_name}__qw'] = q_global[:, j_idx, 3]

df_clean = pd.DataFrame(flattened_data)

# --- 2. Define RUN_ID and output directory ---
RUN_ID = os.path.splitext(os.path.basename(CSV_PATH))[0]
DERIV_01 = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_01_parse")
os.makedirs(DERIV_01, exist_ok=True)

# --- 3. Create THE ULTIMATE ENHANCED REPORT ---
# Enhancement 1: Added calibration data extraction
calibration_data = loader_report.get('calibration', {})

enhanced_report = {
    "identity": {
        "run_id": RUN_ID,
        "processing_timestamp": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M'),
        "pipeline_version": "v2.6_calibration_enhanced",
        "csv_source": CSV_PATH
    },
    "raw_data_quality": {
        "total_frames": len(df_clean),
        "missing_data_percent": loader_report['data_quality']['nan_position_percent'],
        "sampling_rate_actual": loader_report['fps_estimated'],
        "optitrack_mean_error_mm": float(metadata.get('mean_error', 0.0)) if 'metadata' in locals() else 0.0,
        "optitrack_version": calibration_data.get('optitrack_version', 'unknown')
    },
    "calibration": {
        "pointer_tip_rms_error_mm": calibration_data.get('pointer_tip_rms_error_mm'),
        "wand_error_mm": calibration_data.get('wand_error_mm'),
        "export_date": calibration_data.get('export_date')
    },
    "skeleton_info": {
        "segments_found": joints_found,
        "segments_found_count": len(joints_found),
        "segments_missing": loader_report['segments_missing_list'],
        "segments_missing_count": loader_report['segments_missing_count']
    },
    "duration_sec": loader_report['duration_sec']
}

# --- 4. Save report ---
report_path = os.path.join(DERIV_01, f"{RUN_ID}__step01_loader_report.json")
with open(report_path, 'w') as f:
    json.dump(enhanced_report, f, indent=4)

# --- 5. Summary Output ---
print(f"\n{'='*60}")
print(f"✅ STEP 01 COMPLETE: {RUN_ID}")
print(f"📊 Frames: {len(df_clean):,}")
print(f"🎬 Actual FPS: {loader_report['fps_estimated']:.2f}")
print(f"📐 Mean System Error: {enhanced_report['raw_data_quality']['optitrack_mean_error_mm']:.5f} mm")
print(f"🦴 Segments Found: {len(joints_found)}/{loader_report.get('segments_expected', 'N/A')}")
print(f"📂 Audit Report Saved: {report_path}")
print(f"{'='*60}\n")

# Display first few rows
df_clean.head()

# At the end of cells with large arrays, add:
import gc
gc.collect()

# Don't keep multiple copies of large data
# Instead of: df_backup = df_clean
# Use: df_clean.to_parquet('temp_backup.parquet')


✅ STEP 01 COMPLETE: 734_T1_P1_R2_Take 2025-12-01 02.32.02 PM
📊 Frames: 30,514
🎬 Actual FPS: 120.00
📐 Mean System Error: 0.00000 mm
🦴 Segments Found: 51/51
📂 Audit Report Saved: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_01_parse\734_T1_P1_R2_Take 2025-12-01 02.32.02 PM__step01_loader_report.json



63